<a href="https://colab.research.google.com/github/compartia/nlp_tools/blob/tensorflow-model/NER_Matcher_(reg_exps).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
GLOBALS__={}

## MAIN, init, load code

In [0]:
#@title  { form-width: "300px", display-mode: "form" }
import os






# ''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''
import sys
# ====================================
# ====================================
_git_branch = "tensorflow-model"  # @param {type:"string"}
# ====================================
# ====================================


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

  # self-test
  from text_tools import untokenize
  print(untokenize(['code', 'imported', 'OK 👍']))

  print('installing antiword...')
  exec('sudo apt-get install antiword')

  print('installing docx2txt...')
  exec("pip install docx2txt")

  GLOBALS__['_init_import_code_from_gh'] = True

  ''' AZ:-------------------------------------------------IMPORT CODE GITHUB-😺---'''
  print('❤️ DONE importing Code fro GitHub')


# AZ:-INIT ELMO-----------------------------------------------------------------------------------

import tensorflow_hub as hub

#
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML


 

# AZ:- THE CODE----------------------------------------------------------------------------------

def _init_the_code(reset=False):
  if '_init_the_code' in GLOBALS__ and not reset:
    print('👌 Code is alredy imported!')
    return

  from renderer import HtmlRenderer
  from renderer import to_multicolor_text
  from renderer import known_subjects_dict

  from structures import ContractSubject
  from contract_parser import ContractDocument3

  from ml_tools import ProbableValue

  from legal_docs import LegalDocument
  from renderer import as_warning, as_headline_3, as_offset, as_smaller

  class DemoRenderer(HtmlRenderer):

    def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
      display(HTML(html))

    def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

    def render_multicolor_text(self, tokens, vectors, colormap, min_color=None, _slice=None):
      display(HTML(to_multicolor_text(tokens, vectors, colormap, min_color=min_color, _slice=_slice)))

     
     

     
  GLOBALS__['renderer'] = DemoRenderer()

  GLOBALS__['_init_the_code'] = True
  print("❤️ DONE initializing the code")

  # AZ:-------------------------------------------------Init Protocols context===


def read_doc(fn):
  import docx2txt, sys, os

  text = ''
  try:
    text = docx2txt.process(fn)

  except:
    print("Unexpected error:", sys.exc_info())
    os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
    with open(fn + '.txt') as f:
      text = f.read()

  return text

def interactive_upload(filetype):
  from google.colab import files
  import docx2txt

  print(f'Please select "{filetype}" .docx file:')
  uploaded = files.upload()
  docs = []
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

    with open(fn, "wb") as df:
      df.write(uploaded[fn])
      df.close()

    # extract text

    text = ''
    try:
      text = docx2txt.process(fn)
    except:
      print("Unexpected error:", sys.exc_info())
      os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
      with open(fn + '.txt') as f:
        text = f.read()
    print("Символов в оригинальном документе:", len(text))
    docs.append(text)
    return docs

  
# AZ:- ENDO OF THE THE CODE------------------------------------------------XXXX


# 1.
_init_import_code_from_gh()
 
# 3.
_init_the_code(True)



# Batch

### load files from gdrive

In [0]:
# import gspread
from google.colab import auth
from google.colab import drive
# from oauth2client.client import GoogleCredentials

auth.authenticate_user()
# google_spread = gspread.authorize(GoogleCredentials.get_application_default())

drive.mount('/content/gdrive', force_remount=True)


import glob
def read_documents(filename_prefix):
  texts = {}
  for file in glob.glob(filename_prefix+"*.doc"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.doc FILE!!', file)

  for file in glob.glob(filename_prefix+"*.docx"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.docx FILE!!', file)
      
  return texts

contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
contracts = read_documents(contracts_filename_prefix)

In [0]:
BAD_DOCS="""
Договор 5.docx
Проект договора Пур адм.docx
Проект договора Пур адм - благотв.docx
МНПЗ 1 Договор целевого пожертвования ДО ГПН - ФОНД 2019_Умножая таланты_.docx
МНПЗ 2 Договор купли-продажи недвижимого имущества.docx
Доп.согл3 Доверие.doc
Договор выполнения работ_Петров.docx
ДОГОВОР_Детский дом.docx
7. Соглашение БГ Газпромнефть-Региональные продажи.doc
1. Доп. соглашение к Договору_фин.docx
2. Договор по благ-ти Радуга.docx
Договор 4.docx
Договор 3.docx
Договор1.docx
ЕЮ2 Договор пожертвования (одаряемый).docx
ЕЮ2 Договор пожертвования (получатель).docx
ДКП Половинной НБ и АЗС.docx
1.1. Договор пожертвования.docx
"""

BAD_DOCS = [ '/content/gdrive/My Drive/GazpromOil/Contracts/'+x.strip() for x in BAD_DOCS.split('\n') if x.strip()!='' ]
for i in  range(len(BAD_DOCS)):
  print(i, BAD_DOCS[i])

#Reg exp solution

In [0]:
def ru_cap(xx):
  return '\s+'.join(  [ f'[{x[0].upper()}{x[0].lower()}]{x[1:-2]}[а-я]{{0,3}}' for x in xx.split(' ')] )
  
print(ru_cap('Акционерное общество'))

In [0]:
import re


from text_normalize import replacements_regex, normalize_text

def ru_cap(xx):
  return '\s+'.join([f'[{x[0].upper()}{x[0].lower()}]{x[1:-2]}[а-я]{{0,3}}' for x in xx.split(' ')])


q_re_open = '\'\"«'
q_re_close = '\'\"»'

sf = '[а-я]{1,3}'

r_few_words = r'\s+[А-Я]{1}[а-я\-, ]{1,80}\s+'
r_few_words_s = r'\s+[А-Яа-я\-, ]{1,80}\s+'

ORG_TYPES_re = [
  ru_cap('Акционерное общество'), 'АО',
  ru_cap('Закрытое акционерное общество'), 'ЗАО',
  ru_cap('Открытое акционерное общество'), 'ОАО',
  ru_cap('Государственное автономное учреждение')+r_few_words_s,
  ru_cap('Муниципальное бюджетное учреждение'),
  ru_cap('Общественная организация'),
  ru_cap('Общество с ограниченной ответственностью'), 'ООО',
  ru_cap('Некоммерческая организация'),
  ru_cap('организация'),
  ru_cap('Благотворительный фонд'),
  
    
    
  ru_cap('Фонд') + r_few_words_s,
  r_few_words_s + ru_cap('учреждение'),
  ru_cap('Индивидуальный предприниматель'), 'ИП'

]


# ORG_TYPES_re =[ru_cap('Муниципальное бюджетное учреждение')]

r_types_ = '|'.join([x for x in ORG_TYPES_re])
r_types = f'({r_types_})'

r_name_a = r'([–А-Яa-я\- ]{0,30})'
r_name = r'([–А-Яa-я\- ]{0,50})'

r_quote_open = r'([«"<]|[\']{2})'
r_quote_close = r'([»">]|[\']{2})'

r_alter = r'(\s+\(.{1,40}\))?'

complete_re = re.compile(r_types + r'\s*' + r_name_a + r_quote_open + r_name + r_quote_close + r_alter,
                         re.MULTILINE)


def find_org_names(txt):
  def clean(x):
    if x is None: return x
    return x.replace('\t', ' ').replace('\n', ' ').replace(' – ', '-')

  def to_dict(_org):
    return {
      '1_type': clean(_org[1]),
      '2_type_ext': clean(_org[2]),
      '3_name': f'"{clean(_org[4])}"',
      '4_alt_name': clean(_org[6])
    }

  def find_from(start):
    _next = txt[start:]
    r = complete_re.search(_next)
    if r is not None:
      return to_dict(r), start+r.span()[1]
    else: 
      return None, None
    
    
  org_names = {}
  r, end = find_from(0)
  if r is not None:
    org_names[0] = r
    
    r2, end = find_from(end)
    while r2 is not None and r2['3_name'].lower() ==  r['3_name'].lower():
#       print(end)
      r2, end = find_from(end)
    
    if r2 is not None:
      org_names[1] = r2
 
        

  return org_names


##################TEST


def _str_none(x):
  if x is None: 
    return '-'
  else:
    return x

def orgs_to_tsv(org_names)->str:
  ret=''
  if 0 in org_names:   
    ret+=org_to_tsv(org_names[0])
  else:
    ret+='\t'*4
    
  if 1 in org_names:   
    ret+=org_to_tsv(org_names[1])
  else:
    ret+='\t'*4
  return ret
    
def org_to_tsv(org)->str:     
  b = [ _str_none(x) for x in org.values() ]
  return '\t'.join(b)+'\t'
  
  
  

fc=0
export_strings=''
for fn in  contracts:#

  export_string = f'{fn.split("/")[-1]}\t'
  fc += 1
 
  txt = contracts[fn][0:2000]
  txt=normalize_text(txt, replacements_regex)
  org_names = find_org_names(txt)
   
  export_string+=orgs_to_tsv(org_names)
  export_strings+=export_string+'\n'
print(export_strings)